In [ ]:
!pip install beautifulsoup4
!pip install js2xml
!pip install img2pdf


     |████████████████████████████████| 276kB 4.1MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 112kB 4.2MB/s 
     |████████████████████████████████| 2.0MB 7.0MB/s 
  Created wheel for img2pdf: filename=img2pdf-0.4.0-cp36-none-any.whl size=40503 sha256=ed89fe9fede3ee5943391aa86192d9e91950b92125db56c4e720755103dc6b2c
  Stored in directory: /root/.cache/pip/wheels/90/f5/7f/b88a76bac33669118e7549f4856bbe975a3ad2e59280c59196
Successfully built img2pdf


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import re
import img2pdf

In [ ]:
def image_to_pdf(url,pdf_name):
#This function takes an url from a digital file in the Archivo histórico de Medellín and save it as a pdf.
#Extracts urls from web page.
  html_page = urlopen(url)
  soup = BeautifulSoup(html_page, 'html.parser')
  x=soup.findAll(name="script")#Get all script tags (the page has a javascript app).
  values = re.findall(r'jzd_pages..*?=\s*(.*?);', str(x), re.DOTALL | re.MULTILINE)[0]#Get the matrix with images extensions
  x2=values.split('Array(')[1].split(')')[0]
  semi_url=[x2.replace('"', '') for x2 in x2.split(",")]#Get the images extensions clean.
#Getting the pieces of the urls.
  dir=re.findall('(?<=jzd_current_print_image\(\)).*?(?=jzd_current_image)', str(x), re.DOTALL | re.MULTILINE)[0]#Entire url from a function without matrix extensions.
  dir_sub1=re.findall("(?<=Documentos/).*?(?=\'\s*\+)",dir)[0]#First part of the url.
  dir_sub2=re.findall("(?<=\+\s\').*?(?=\'\s*\+)",dir)[0]
  dir_sub3=re.findall("(?<=\+\s\').*(?=\';)",dir)[0]
  urls_list=[]
  for index,value in enumerate(semi_url):
    #Build the url.
    urls_list.append("https://ahmedellin.janium.net/janium/Documentos/{}{}{}{}".format(dir_sub1,dir_sub2,semi_url[index],dir_sub3))
#Download the images.
  imgs_pdf=[]
  for img in urls_list:
    file_name = img.split('/')[-1]#Get only the names
    imgs_pdf.append(str(file_name))#Save them on a vector
    r = requests.get(img, stream=True)#Download files.
    with open(file_name, 'wb') as f:
     #open the files and save them. 
      for chunk in r:
        f.write(chunk)
#Save as pdf using img2pdf library.
  with open(pdf_name,"wb") as f:
	  f.write(img2pdf.convert(imgs_pdf))



In [ ]:
image_to_pdf("https://ahmedellin.janium.net/janium-bin/janium_zui.pl?fn=400782&jzd=/janium/Documentos/AP/400782/d.jzd","Censo (1851).pdf")
